### Purpose

Test the WaveNet network on `pseudoGR` features. Demonstrate the difference between `categorical` loss, `ordinal` loss, and a combination.

In [1]:
import numpy as np
from sklearn.metrics import confusion_matrix

from coremdlr.viz import model_plots
from coremdlr.models import NetworkModel
from coremdlr.datasets import DepthSequenceGenerator, FaciesDataset

import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
well_names = ["205-21b-3", "204-20-6a", "204-24a-6", "204-20-1Z", "204-19-6", "204-20-1"]
#well_names = [ "204-19-7", "204-20-1","204-20-1Z","204-20-2","204-20-3", "205-21b-3", "204-24a-7" ]

loss_options = ('categorical_crossentropy',
                'ordinal_squared_error',
                {'categorical_crossentropy': 0.2,
                 'ordinal_squared_error': 0.8})

wavenet_model_args = {
    'feature' : 'pseudoGR',
    'network' : 'wavenet',
    'sequence_size' : 128,
    'network_args' : {
        'num_blocks' : 20,
        'block_filters' : 128,
        'residual_filters' : 64,
        'output_resolution' : 32,
        'dropout_rate' : 0.5,
    },
    'optimizer_args' : {
        'optimizer' : 'Adam'
    }
}

fit_args = {
    'batch_size' : 16,
    'epochs' : 15,
    'class_weighted': True
}


In [3]:
np.type(well_names)

AttributeError: module 'numpy' has no attribute 'type'

In [ ]:
for test_well in well_names:
    
    print(f'\n\nTESTING WITH TEST WELL: {test_well}\n\n')
    
    fdset = FaciesDataset([name for name in well_names if name != test_well],
                    test_wells=[test_well],
                    features=["pseudoGR"],
                    label_resolution=32, 
                    pseudoGR_args={'scale_mode' : 'robust'})

    fdset.load_or_generate_data()
    
    for loss in loss_options:
        
        print(f'\nMODELING WITH LOSS: {loss}\n')
        
        model_args = {**wavenet_model_args, **{'loss': loss}}
        
        wavenet_model = NetworkModel(fdset, model_args=wavenet_model_args)
    
        wavenet_model.fit(fdset, **fit_args)
        
        y_pred = wavenet_model.predict(fdset.X_test)

        cm = confusion_matrix(fdset.y_test, y_pred)
        
        fig, ax = plt.subplots(ncols=2, sharey=True, figsize=(20,10))
        fig.suptitle(f'{test_well}: {loss}')

        model_plots.confusion_matrix_plot(cm, fdset.classes, ax=ax[0])
        model_plots.confusion_matrix_plot(cm, fdset.classes, ax=ax[1], normalize=True)
        
        plt.show()



TESTING WITH TEST WELL: 205-21b-3


Loading Well:  204-20-6a  from  /home/administrator/Dropbox/core_data/facies/train_data


/home/administrator/code/python/coremdlr/coremdlr/datasets/utils.py:79: RuntimeWarning: Mean of empty slice
  output_features.append(np.nanmean(img, axis=1))
/home/administrator/code/python/coremdlr/coremdlr/datasets/utils.py:83: RuntimeWarning: Degrees of freedom <= 0 for slice.
  output_features.append(np.nanvar(img, axis=1))


Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (3542,)), ('top', (3542,)), ('base', (3542,)), ('pseudoGR', (3542, 32, 8))]
Loading Well:  204-24a-6  from  /home/administrator/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (13006,)), ('top', (13006,)), ('base', (13006,)), ('pseudoGR', (13006, 32, 8))]
Loading Well:  204-20-1Z  from  /home/administrator/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (1917,)), ('top', (1917,)), ('base', (1917,)), ('pseudoGR', (1917, 32, 8))]
Loading Well:  204-19-6  from  /home/administrator/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (1884,)), ('top',

/home/administrator/code/python/coremdlr/coremdlr/datasets/utils.py:79: RuntimeWarning: Mean of empty slice
  output_features.append(np.nanmean(img, axis=1))
/home/administrator/code/python/coremdlr/coremdlr/datasets/utils.py:83: RuntimeWarning: Degrees of freedom <= 0 for slice.
  output_features.append(np.nanvar(img, axis=1))


Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (3842,)), ('top', (3842,)), ('base', (3842,)), ('pseudoGR', (3842, 32, 8))]

MODELING WITH LOSS: categorical_crossentropy

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 4096, 8)      0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 4096, 64)     3648        input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 4096, 128)    24704       conv1d[0][0]                     
______________________________________________________________

In [4]:
# What's up with this well -- all sand?
from coremdlr.viz import CorePlotter

fdset = Facies

well = fdset.get_well("204-20-1Z")

cplotter = CorePlotter(figsize=(10,200))
cplotter.plot_image(well._image / 255.0, depths=well._depth)

labels = well.make_striplog()
cplotter.plot_strip(labels)

plt.show()

NameError: name 'Facies' is not defined

## Categorical Loss

In [ ]:
well_names = ["205-21b-3", "204-20-6a", "204-20-1Z", "204-19-6", "204-24a-6"]

fdset = FaciesDataset(well_names[:-1], [well_names[-1]], features=['pseudoGR'], label_resolution=32)

fdset.load_or_generate_data()

In [ ]:
wavenet_model_args = {
    'feature' : 'pseudoGR',
    'network' : 'wavenet',
    'summary' : False,
    'sequence_size' : 128,
    'network_args' : {
        'num_blocks' : 20,
        'block_filters' : 128,
        'residual_filters' : 64,
        'output_resolution' : fdset.label_resolution,
        'dropout_rate' : 0.5,
    },
    'optimizer_args' : {
        'optimizer' : 'Adam',
    },
    'loss' : 'categorical_crossentropy'
}

fit_args = {
    'batch_size' : 32,
    'epochs' : 15,
    'step_size' : wavenet_model.sequence_size - 32
}



In [ ]:
fit_args = {
    'batch_size' : 32,
    'epochs' : 15,
    'step_size' : wavenet_model.sequence_size - 32
}

wavenet_model.fit(fdset, **fit_args)

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = wavenet_model.predict(fdset.X_test)

cm = confusion_matrix(fdset.y_test, y_pred)

model_plots.confusion_matrix_plot(cm, fdset.classes)

## Ordinal Loss

In [ ]:
wavenet_model_args['loss'] = 'ordinal_squared_error'

wavenet_model = NetworkModel(fdset, model_args=wavenet_model_args)

wavenet_model.fit(fdset, **fit_args)

In [ ]:
y_pred = wavenet_model.predict(fdset.X_test)

cm = confusion_matrix(fdset.y_test, y_pred)

model_plots.confusion_matrix_plot(cm, fdset.classes)

## Combined Loss

In [ ]:
wavenet_model_args['loss'] = {'categorical_crossentropy': 0.15, 
                              'ordinal_squared_error' : 0.85}

wavenet_model = NetworkModel(fdset, model_args=wavenet_model_args)

In [ ]:
wavenet_model.fit(fdset, **fit_args)

In [ ]:
y_pred = wavenet_model.predict(fdset.X_test)

cm = confusion_matrix(fdset.y_test, y_pred)

model_plots.confusion_matrix_plot(cm, fdset.classes)

## Try same with class weighting

In [ ]:
fit_args['class_weighted'] = True
fit_args['epochs'] = 9

wavenet_model = NetworkModel(fdset, model_args=wavenet_model_args)

wavenet_model.fit(fdset, **fit_args)

In [ ]:
y_pred = wavenet_model.predict(fdset.X_test)

cm = confusion_matrix(fdset.y_test, y_pred)

model_plots.confusion_matrix_plot(cm, fdset.classes)

In [ ]:
np.bincount(fdset.y_train) / fdset.y_train.size

In [ ]:
fit_args['class_weighted'] = True
fit_args['epochs'] = 30

wavenet_model = NetworkModel(fdset, model_args=wavenet_model_args)

wavenet_model.fit(fdset, **fit_args)